### Import Library

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.dates as md
from matplotlib import pyplot as plt
import pickle

import torch
import torch.nn as nn
import torch.utils.data as data_utils
import random

### get_seed

In [4]:
random_seed = 42
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

np.random.seed(random_seed)
random.seed(random_seed)

torch.cuda.manual_seed(random_seed)

### get_array

In [6]:
train_array = np.load('/workspace/대회/npy_files/all_array_.npy')
inf_array = np.load('/workspace/대회/npy_files/inf_array_.npy')
print(train_array.shape)
print(inf_array.shape)

(58631, 2000, 10)
(8744, 2000, 10)


In [7]:
BATCH_SIZE =  128

train_loader = torch.utils.data.DataLoader(train_array[:300,:,:3], batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
inf_loader = torch.utils.data.DataLoader(inf_array[:,:,:3], batch_size = BATCH_SIZE, shuffle=False, num_workers=0)

### Encoder

In [8]:
class Encoder(nn.Module):
    def __init__(self,  input_size, hidden_size_1, hidden_size_2, num_layers = 1):
        super().__init__()
        self.input_size = input_size #input size
        self.hidden_size_1 = hidden_size_1 #hidden state == output_vector
        self.hidden_size_2 = hidden_size_2 #hidden state == output_vector
        self.num_layers = num_layers #number of layers == 몇층


        self.lstm_1 = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size_1,
                      num_layers=self.num_layers, batch_first=True)

        self.lstm_2 = nn.LSTM(input_size=self.hidden_size_1, hidden_size=self.hidden_size_2,
                      num_layers=self.num_layers, batch_first=True)


    def forward(self, w):
        out_1, _ = self.lstm_1(w)

        return self.lstm_2(out_1)

### Decoder

In [9]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_layers = 1):
        super().__init__()
        self.input_size = input_size #input size
        self.hidden_size_1 = hidden_size_1 #hidden state == output_vector
        self.hidden_size_2 = hidden_size_2 #hidden state == output_vector
        self.num_layers = num_layers #number of layers == 몇층

        self.lstm_1 = nn.LSTM(input_size=self.hidden_size_2, hidden_size=self.hidden_size_1,
                      num_layers=self.num_layers, batch_first=True)
        
        self.lstm_2 = nn.LSTM(input_size=self.hidden_size_1, hidden_size=self.input_size,
                      num_layers=self.num_layers, batch_first=True)


    def forward(self, z):
        out_1, _ = self.lstm_1(z)

        return self.lstm_2(out_1)

### Model_

In [10]:
class UsadModel_LSTM_AE(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_layers = 1):
        super().__init__()
        self.input_size = input_size #input size
        self.hidden_size_1 = hidden_size_1 #hidden state == output_vector
        self.hidden_size_2 = hidden_size_2 #hidden state == output_vector
        self.num_layers = num_layers #number of layers == 몇층
        
        self.encoder = Encoder(input_size, hidden_size_1, hidden_size_2, num_layers = 1)
        self.decoder1 = Decoder(input_size, hidden_size_1, hidden_size_2, num_layers = 1)
        
    def forward(self, x, n):
        out_, _ = self.encoder(x)        
        out_.to(device)
        w1, _= self.decoder1(out_)
        w1.to(device)
        
        mse_ = (x-w1)**2

        return mse_, w1, out_
    
device = 'cuda'
model_lstam_ae = UsadModel_LSTM_AE(3,32,16)
optimizer = torch.optim.Adam(list(model_lstam_ae.encoder.parameters())+list(model_lstam_ae.decoder1.parameters()))
model_lstam_ae.to(device)

UsadModel_LSTM_AE(
  (encoder): Encoder(
    (lstm_1): LSTM(3, 32, batch_first=True)
    (lstm_2): LSTM(32, 16, batch_first=True)
  )
  (decoder1): Decoder(
    (lstm_1): LSTM(16, 32, batch_first=True)
    (lstm_2): LSTM(32, 3, batch_first=True)
  )
)

### train_function

In [11]:
def train(model, train_loader, epoch, optimizer, device='cuda'):

    model.train()

    losses_train = []
    for batch in train_loader:
        mse_, w1, out_ = model(batch.type(torch.FloatTensor).to(device),epoch+1)
        loss = torch.mean(mse_)
        loss.backward(retain_graph=True)
        optimizer.step()
        optimizer.zero_grad()
        losses_train.append(loss.item())

    losses_train = np.mean(losses_train)


    return losses_train

In [12]:
epochs = 5
for epoch in range(epochs):
    train_loss_ = train(model_lstam_ae, train_loader, epochs, optimizer)
    print(f'epoch - {epoch}  train_loss_1 - {train_loss_}')

epoch - 0  train_loss_1 - 24.719334602355957
epoch - 1  train_loss_1 - 24.66745726267497
epoch - 2  train_loss_1 - 24.60614077250163
epoch - 3  train_loss_1 - 24.529312133789062
epoch - 4  train_loss_1 - 24.431960423787434


### Inference_testing

In [13]:
def testing(model, test_loader, device = 'cuda'):
    results = np.empty([0, 3])
    with torch.no_grad():
        for batch in test_loader:
            mse_, w1, out_, = model(batch.type(torch.FloatTensor).to(device),epoch+1)
            mse_ = mse_.cpu()
            w1 = w1.cpu()
            out_ = out_.cpu()
            mse_ = np.array(mse_)
            sum_mse = mse_.sum(axis = 1)
            results = np.concatenate([results,sum_mse])
        return results

In [15]:
# results = testing(model_lstam_ae,inf_loader)
results

array([[10461.53710938, 10381.47167969, 10819.54785156],
       [20037.90429688, 21167.50390625, 18207.08203125],
       [10606.8046875 , 10529.0390625 , 11033.57226562],
       ...,
       [14048.08398438, 14981.203125  , 14004.52050781],
       [13712.1875    , 14330.04003906, 13644.75878906],
       [31654.55859375, 35891.390625  , 39958.26953125]])